In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [88]:
# Parameters
S0 = 75                             # Initial stock price
F = 100                             # Face value of the bond
X = 100                             # Strike price for conversion
T = 5                               # Time to maturity (in years)
r = 0.07                            # Risk-free rate
q = 0
b = r - q
k = 0.03                            # Credit spread
sigma = 0.2                         # Volatility
dt = 1
coupon = 6                          # Coupon payment per year
m = 1                               # Conversion ratio (1 bond = 1 stock)
N = 5                               # Number of time steps
conversion_ratio = F / X            # Conversion ratio

# Convertible bond pricing model (Cox-Ross-Rubinstein method)
def CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e'):
    # Time step
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))     # Up factor
    d = 1 / u                           # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Create the binomial stock price tree
    StockPrice = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            StockPrice[j, i] = S0 * (u ** j) * (d ** (i - j))

    # Create the bond price tree (including coupon payments)
    BondPrice = np.zeros((N + 1, N + 1))
    # Create the conversion probability tree
    ConversionProbability = np.zeros((N + 1, N + 1))

    # Calculate bond prices and conversion probabilities at maturity
    for j in range(N + 1):
        stock_price = StockPrice[j, N]
        bond_value_at_maturity = max(conversion_ratio * stock_price, F + coupon)
        BondPrice[j, N] = bond_value_at_maturity
        # Conversion probability is 1 if conversion is optimal at maturity
        ConversionProbability[j, N] = 1 if conversion_ratio * stock_price > F + coupon else 0

    # Initialize coupon timing and time management
    CouponTime = T  # Initial coupon time matches maturity

    # Backward calculation for bond price and conversion probability
    for i in range(N - 1, -1, -1):
        if CouponTime != (T - np.ceil((N - i + 1) * dt)):
            CouponPayment = 1  # It's time to pay a coupon
            CouponTime = T - np.ceil((N - i) * dt)
            Time = CouponTime - dt * i
        else:
            CouponPayment = 0

        for j in range(i + 1):
            stock_price = StockPrice[j, i]

            # Dynamic discount factors (ru, rd)
            ru = ConversionProbability[j+1, i+1] * r + (1 - ConversionProbability[j+1, i+1]) * (r + k)
            rd = ConversionProbability[j, i+1] * r + (1 - ConversionProbability[j, i+1]) * (r + k)

            if CouponPayment == 1:
                CouponValue = coupon * (p * np.exp(-ru * Time) + (1 - p) * np.exp(-rd * Time))
            else:
                CouponValue = 0

            # Calculate bond price using backward induction
            hold_bond_value = CouponValue + p * BondPrice[j + 1, i + 1] * np.exp(-ru * dt) + (1 - p) * BondPrice[j, i + 1] * np.exp(-rd * dt)
            convert_to_stock_value = conversion_ratio * stock_price

            # Modify for American or European option style
            if AmeEur == 'a':
                # American style: check for early exercise
                BondPrice[j, i] = max(hold_bond_value, convert_to_stock_value)
            else:
                # European style: only exercise at maturity
                BondPrice[j, i] = hold_bond_value

            # Conversion probability: set to 1 if conversion happens, otherwise average the next nodes
            if BondPrice[j, i] == convert_to_stock_value and AmeEur == 'a':
                ConversionProbability[j, i] = 1
            else:
                ConversionProbability[j, i] = p * ConversionProbability[j + 1, i + 1] + (1 - p) * ConversionProbability[j, i + 1]

    return BondPrice[0,0]



In [ ]:
# Delta Calculation
def calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump=0.01):
    bond_price_up = CRR_convertible_bond(S0 + (S0*bump), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0 - (S0*bump), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    # Calculate delta
    delta = (bond_price_up - bond_price_down) / (2 * bump * 100)
    return delta

# Calculate Delta
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Delta of the Convertible Bond: {delta:.4f}")


Delta of the Convertible Bond: 0.4056


In [ ]:
def calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_rho=1/10000):
    bond_price_up = CRR_convertible_bond(S0, F, T, r + bump_rho, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0, F, T, r - bump_rho, k, coupon, conversion_ratio, N, sigma, AmeEur)
    # Calculate Rho
    rho = (bond_price_up - bond_price_down) / (2 * bump_rho)
    return rho

rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Rho of the Convertible Bond: {rho:.4f}")

Rho of the Convertible Bond: -189.6374


In [ ]:
# Function to calculate Vega
def calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_sigma=0.0001):
    bond_price_up = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma + bump_sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma - bump_sigma, AmeEur)
    vega = (bond_price_up - bond_price_down) / (2 * bump_sigma)
    return vega

# Calculate Vega
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Vega of the Convertible Bond: {vega:.4f}")


Vega of the Convertible Bond: 35.7933


In [ ]:
# Function to calculate Gamma
def calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_gamma=0.001):
    bond_price_up = CRR_convertible_bond(S0 + (S0*bump_gamma), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_mid = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0 - (S0*bump_gamma), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    # Calculate Gamma
    gamma = (bond_price_up - (2 * bond_price_mid) + bond_price_down) / ((S0*bump_gamma) ** 2)
    return gamma

# Calculate Gamma
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Gamma of the Convertible Bond: {gamma:.4f}")

Gamma of the Convertible Bond: 0.0000


In [ ]:
# Function to calculate Theta
def calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_theta=1/360):
    bond_price_shorter_time = CRR_convertible_bond(S0, F, T - bump_theta, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_current_time = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    theta = (bond_price_shorter_time - bond_price_current_time) / bump_theta
    return theta

# Calculate Theta
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Theta of the Convertible Bond: {theta:.4f}")


Theta of the Convertible Bond: 4.8487


In [167]:
S0 = 75                             # Initial stock price
F = 100                             # Face value of the bond
X = 100                             # Strike price for conversion
T = 5                               # Time to maturity (in years)
t1= 0
r = 0.07                            # Risk-free rate
q = 0
b = r - q
k = 0.03                            # Credit spread
sigma = 0.2                         # Volatility
dt = 1
coupon = 6                          # Coupon payment per year
m = 1                               # Conversion ratio (1 bond = 1 stock)
N = 5                              # Number of time steps
conversion_ratio = F / X            # Conversion ratio

# Function to calculate tree parameters (u, d, p) for different models
def calculate_tree_parameters(r, q, sigma, dt, model_type, drift=0):
    if model_type == 'CRR':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(r * dt) - d) / (u - d)
    elif model_type == 'CRR_Drift':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp((r + drift) * dt) - d) / (u - d)
    elif model_type == 'Jarrow-Rudd':
        u = np.exp((r - q - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt))
        d = np.exp((r - q - 0.5 * sigma ** 2) * dt - sigma * np.sqrt(dt))
        p = 0.5
    elif model_type == 'Tian':
        eta=np.exp(dt*sigma**2)
        u=0.5*np.exp(r*dt)*eta*(eta+1+np.sqrt(eta**2+2*eta-3))
        d=0.5*np.exp(r*dt)*eta*(eta+1-np.sqrt(eta**2+2*eta-3))
        p=(np.exp((b)*dt)-d)/(u-d)
    elif model_type == 'Haahtela':
        u = np.exp(r * dt) * (1 + np.sqrt(np.exp((sigma ** 2) * dt) - 1))
        d = np.exp(r * dt) * (1 - np.sqrt(np.exp((sigma ** 2) * dt) - 1))
        p = (np.exp((r - q) * dt) - d) / (u - d)
    else:
        raise ValueError("Unknown model type provided.")

    return u, d, p

def LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='', AmeEur='e', q=0, drift=0):
    dt = T / N  # Time step
    sigma = 0.2  # Assume a constant volatility for demonstration purposes

    # Calculate u, d, p using the specified model type
    u, d, p = calculate_tree_parameters(r, q, sigma, dt, model_type, drift)

    # Create the binomial stock price tree
    StockPrice = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            StockPrice[j, i] = S0 * (u ** j) * (d ** (i - j))

    # Create the bond price tree (including coupon payments)
    BondPrice = np.zeros((N + 1, N + 1))
    # Create the conversion probability tree
    ConversionProbability = np.zeros((N + 1, N + 1))

    # Calculate bond prices and conversion probabilities at maturity
    for j in range(N + 1):
        stock_price = StockPrice[j, N]
        bond_value_at_maturity = max(conversion_ratio * stock_price, F + coupon)
        BondPrice[j, N] = bond_value_at_maturity
        ConversionProbability[j, N] = 1 if conversion_ratio * stock_price > F + coupon else 0

    # Initialize coupon timing and time management
    CouponTime = T  # Initial coupon time matches maturity

    # Backward calculation for bond price and conversion probability
    for i in range(N - 1, -1, -1):
        if CouponTime != (T - np.ceil((N - i + 1) * dt)):
            CouponPayment = 1  # It's time to pay a coupon
            CouponTime = T - np.ceil((N - i) * dt)
            Time = CouponTime - dt * i
        else:
            CouponPayment = 0

        for j in range(i + 1):
            stock_price = StockPrice[j, i]

            # Dynamic discount factors (ru, rd)
            ru = ConversionProbability[j+1, i+1] * r + (1 - ConversionProbability[j+1, i+1]) * (r + k)
            rd = ConversionProbability[j, i+1] * r + (1 - ConversionProbability[j, i+1]) * (r + k)

            if CouponPayment == 1:
                CouponValue = coupon * (p * np.exp(-ru * Time) + (1 - p) * np.exp(-rd * Time))
            else:
                CouponValue = 0

            hold_bond_value = CouponValue + p * BondPrice[j + 1, i + 1] * np.exp(-ru * dt) + (1 - p) * BondPrice[j, i + 1] * np.exp(-rd * dt)
            convert_to_stock_value = conversion_ratio * stock_price

            # Modify for American or European option style
            if AmeEur == 'a':  # American option: can convert at any time
                BondPrice[j, i] = max(hold_bond_value, convert_to_stock_value)

            else:  # European option: can only convert at maturity
                BondPrice[j, i] = hold_bond_value

            # Update the conversion probability based on the style
            if AmeEur == 'a' and BondPrice[j, i] == convert_to_stock_value:
                ConversionProbability[j, i] = 1
            else:
                ConversionProbability[j, i] = p * ConversionProbability[j + 1, i + 1] + (1 - p) * ConversionProbability[j, i + 1]

    return BondPrice


In [194]:
# Define all model types

# Function to calculate Delta Greek for all models
def calculate_delta_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='a', q=0, drift=0, bump=0.01):
    model_types = ['CRR', 'CRR_Drift', 'Jarrow-Rudd', 'Tian', 'Haahtela']
    delta_results = {}
    for model in model_types:
        # Calculate Delta for the current model type
        bond_price_up = LatticeConvertibleBond(S0 + (S0*bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        bond_price_down = LatticeConvertibleBond(S0 - (S0*bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        delta = (bond_price_up - bond_price_down) / (2 * bump * 100)
        delta_results[model] = delta    # Store the Delta in the results 
    return delta_results


# Calculate Delta for all models
deltas_models = calculate_delta_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma)

print("Delta Greek for All Models:")
for model, delta in deltas_models.items():
    print(f"{model}: {delta:.4f}")


Delta Greek for All Models:
CRR: 0.4056
CRR_Drift: 0.4056
Jarrow-Rudd: 0.4927
Tian: 2.6432
Haahtela: 0.4960


In [193]:
# Function to calculate Rho Greek for all models
def calculate_rho_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='a', q=0, drift=0, bumb_rho=0.0001):
    model_types = ['CRR', 'CRR_Drift', 'Jarrow-Rudd', 'Tian', 'Haahtela']
    rho_results = {}
    for model in model_types:
        bond_price_up = LatticeConvertibleBond(S0, F, T, r + bumb_rho, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        bond_price_down = LatticeConvertibleBond(S0, F, T, r - bumb_rho, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        # Calculate Rho
        rho = (bond_price_up - bond_price_down) / (2 * bumb_rho)
        rho_results[model] = rho
    return rho_results

# Calculate Rho for all models
rho_models = calculate_rho_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma)

print("Rho Greek for All Models:")
for model, rho in rho_models.items():
    print(f"{model}: {rho:.4f}")

Rho Greek for All Models:
CRR: -189.6374
CRR_Drift: -189.6374
Jarrow-Rudd: -213.6300
Tian: -470.2930
Haahtela: -213.6300


In [195]:
model_types = ['CRR', 'CRR_Drift', 'Jarrow-Rudd', 'Tian', 'Haahtela']
# Function to calculate Gamma Greek for all models
def calculate_gamma_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='a', q=0, drift=0, bump_gamma=0.01):
    gamma_results = {}
    for model in model_types:
        bond_price_up = LatticeConvertibleBond(S0 + bump_gamma, F, T, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        bond_price_mid = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        bond_price_down = LatticeConvertibleBond(S0 - bump_gamma, F, T, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        # Calculate Gamma
        gamma = (bond_price_up - (2 * bond_price_mid) + bond_price_down) / ((S0*bump_gamma) ** 2)
        gamma_results[model] = gamma
    return gamma_results

# Calculate Gamma for all models
gamma_results = calculate_gamma_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma)

# Display the results
print("Gamma Greek for All Models:")
for model, gamma in gamma_results.items():
    print(f"{model}: {gamma:.4f}")


Gamma Greek for All Models:
CRR: 0.0000
CRR_Drift: 0.0000
Jarrow-Rudd: 0.0000
Tian: -0.0000
Haahtela: -0.0000


In [196]:
model_types = ['CRR', 'CRR_Drift', 'Jarrow-Rudd', 'Tian', 'Haahtela']
def calculate_vega_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='a', q=0, drift=0, bump_sigma=0.0001):
    vega_results = {}
    for model in model_types:
        bond_price_up = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma + bump_sigma, model, AmeEur, q, drift)[0, 0]
        bond_price_down = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma - bump_sigma, model, AmeEur, q, drift)[0, 0]
        # Calculate Vega
        vega = (bond_price_up - bond_price_down) / (2 * bump_sigma)
        vega_results[model] = vega
    return vega_results

# Calculate Vega for all models
vega_results = calculate_vega_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma)

print("Vega Greek for All Models:")
for model, vega in vega_results.items():
    print(f"{model}: {vega:.4f}")


Vega Greek for All Models:
CRR: 0.0000
CRR_Drift: 0.0000
Jarrow-Rudd: 0.0000
Tian: 0.0000
Haahtela: 0.0000


In [197]:
# Function to calculate Theta Greek for all models
def calculate_theta_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='a', q=0, drift=0, bump_theta=1/360):
    model_types = ['CRR', 'CRR_Drift', 'Jarrow-Rudd', 'Tian', 'Haahtela']
    theta_results = {}
    for model in model_types:
        bond_price_shorter_time = LatticeConvertibleBond(S0, F, T - bump_theta, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        bond_price_current_time = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model, AmeEur, q, drift)[0, 0]
        # Calculate Theta
        theta = (bond_price_shorter_time - bond_price_current_time) / bump_theta
        theta_results[model] = theta
    return theta_results

# Calculate Theta for all models
theta_results = calculate_theta_models(S0, F, T, r, k, coupon, conversion_ratio, N, sigma)

print("Theta Greek for All Models:")
for model, theta in theta_results.items():
    print(f"{model}: {theta:.4f}")

Theta Greek for All Models:
CRR: 4.8487
CRR_Drift: 4.8487
Jarrow-Rudd: 4.3754
Tian: 3.9328
Haahtela: 4.2794
